In [ ]:
import json
import os
from os.path import isdir
import pandas as pd
import geopandas as gpd
from pyproj import CRS

from osmnx.geocoder import geocode_to_gdf

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

import sharedstreets.dataframe

In [ ]:
# project directory
root_dir = os.path.join(r"C:\Users\uscd675041\WSP O365\BART-TDLU - Documents\Networks")

# software directory
ranch_dir = os.path.join("c:/apps/cowboy/Ranch")

In [ ]:
parameters = Parameters(ranch_base_dir = ranch_dir)

In [ ]:
counties = pd.read_csv(os.path.join(root_dir, '..', 'data', 'counties.csv'))

In [ ]:
for county_name in list(counties['COUNTY_NAME']):
    alias = county_name.replace(' ', '')
    exists = False
    
    for ext in ['', f'_{alias}']:
        link_file_name = os.path.join(root_dir, alias, 'interim', f'step3_links{ext}.geojson')
        node_file_name = os.path.join(root_dir, alias, 'interim', f'step3_nodes{ext}.geojson')
        shape_file_name = os.path.join(root_dir, alias, 'interim', f'step3_shapes{ext}.geojson')
        if os.path.exists(link_file_name):
            exists = True
            break
    
    print(f'{alias}:{exists}')
    
    if not exists:
        continue

    # this is the input county polygon file with county names
    print(f'Retrieve County Boundary: {county_name}')
    if county_name == 'San Francisco': # Because San Francisco is a city and a county / OSMNX gets confused.
        cnty = geocode_to_gdf(f"{county_name}, California, USA").to_crs(epsg=4326)
    else:
        cnty = geocode_to_gdf(f"{county_name} County, California, USA").to_crs(epsg=4326)
    cnty.to_file(os.path.join(root_dir, alias, f'cnty_boundary_{alias}.shp'))

    county_boundary_file = os.path.join(root_dir, alias, f'cnty_boundary_{alias}.shp')

    # specify the column to look for county name
    county_variable_name = 'display_na'

    print('Loading Links')
    with open(os.path.join(link_file_name), 'r') as f:
        j = json.load(f)
        links_gdf = gpd.GeoDataFrame.from_features(j)
    links_gdf.crs = 'epsg:4326'

    print('Loading Nodes')
    with open(os.path.join(node_file_name), 'r') as f:
        j = json.load(f)
        node_gdf = gpd.GeoDataFrame.from_features(j)
    node_gdf.crs = 'epsg:4326'

    print('Loading Shapes')
    with open(os.path.join(shape_file_name), 'r') as f:
        j = json.load(f)
        shape_gdf = gpd.GeoDataFrame.from_features(j)
    shape_gdf.crs = 'epsg:4326'

    roadway_network = Roadway(node_gdf, links_gdf, shape_gdf, parameters=parameters)

    roadway_network.tidy_roadway(
        county_boundary_file = county_boundary_file,
        county_variable_name = county_variable_name
    )

    RanchLogger.info("write out shape geojson")

    shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
    shape_geojson = link_df_to_geojson(
        roadway_network.shapes_df, 
        shape_prop
    )

    with open(os.path.join(root_dir, alias, f"step8_shapes_{alias}.geojson"), "w") as f:
        json.dump(shape_geojson, f)

    RanchLogger.info("write out node geojson")

    node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
    node_geojson = point_df_to_geojson(
        roadway_network.nodes_df, 
        node_prop
    )

    with open(os.path.join(root_dir, alias, f"step8_nodes_{alias}.geojson"), "w") as f:
        json.dump(node_geojson, f)

    RanchLogger.info("write out link geojson")

    roadway_network.links_df['COUNTY'] = county_name
    link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
    link_geojson = link_df_to_geojson(
        roadway_network.links_df, 
        link_prop
    )

    with open(os.path.join(root_dir, alias, f"step8_links_{alias}.geojson"), "w") as f:
        json.dump(link_geojson, f)